In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
import tools

In [ ]:
experiments = tools.get_all_experiments()

In [ ]:
def included_reference(info):
    if info['disable_multicast'] == 'True':
        return True
    return False

def included_multicast_experiments_no_fec(info):
    if info['disable_multicast'] == 'True':
        return False
    return info['fec'] == '0'


def included_multicast_experiments_fec(info):
    if info['disable_multicast'] == 'True':
        return False
    return info['fec'] != '0'


def get(experiments, events, diff=False, bytes_to_bits=True):
    df = tools.get_metrics_by_events_from_experiments(experiments, events, align=True, resample='250ms')

    if diff:
        df = df.diff() / 0.25  # 0.25 seconds (250ms) is the time interval

    if bytes_to_bits:
        df = df * 8  # 1 byte = 8 bits

    # Set negative values to NaN
    df = df.mask(df < 0)

    # Extract common parts of column names
    df.columns = df.columns.str.extract(r'(\d+)\s(.+?)(_\d+)?\s(.+)')

    #df = df[[col for col in df.columns if not np.isnan(col[0])]]

    def get_fec(column):
        return experiments[int(column)]['info']['fec']

    def get_loss(column):
        return '{:04.1f}'.format(float(experiments[int(column)]['info']['loss']))

    # Rename columns
    df.columns = [f"{'{:02d}'.format(int(col[0]))} - {get_loss(col[0])} - {get_fec(col[0])} - {col[1]}" for col in df.columns]

    # Group the columns with the same name together and calculate the mean
    # This takes the mean of the proxies of one experiment
    df = df.T.groupby(level=0).mean().T

    return df

def filter(experiments, df, should_include_func):
    def should_include(column):
        parts = column.split(' ')
        if len(parts) == 0:
            return False
        i = int(parts[0])
        return should_include_func(experiments[i]['info'])
    return df[[col for col in df.columns if should_include(col)]]

In [ ]:
df_rx = get(experiments, ['bytes_received']) / 1000000
df_rx_reference = filter(experiments, df_rx, should_include_func=included_reference)
df_rx_no_fec = filter(experiments, df_rx, should_include_func=included_multicast_experiments_no_fec)
df_rx_fec = filter(experiments, df_rx, should_include_func=included_multicast_experiments_fec)

df_tx = get(experiments, ['bytes_transmitted']) / 1000000
df_tx_reference = filter(experiments, df_tx, should_include_func=included_reference)
df_tx_no_fec = filter(experiments, df_tx, should_include_func=included_multicast_experiments_no_fec)
df_tx_fec = filter(experiments, df_tx, should_include_func=included_multicast_experiments_fec)

df_rx_proxy = df_rx.filter(like='proxy') 
df_rx_proxy_reference = df_rx_reference.filter(like='proxy') 
df_rx_proxy_no_fec = df_rx_no_fec.filter(like='proxy') 
df_rx_proxy_fec = df_rx_fec.filter(like='proxy')

df_tx_server = df_tx.filter(like='server') 
df_tx_server_reference = df_tx_reference.filter(like='server') 
df_tx_server_no_fec = df_tx_no_fec.filter(like='server') 
df_tx_server_fec = df_tx_fec.filter(like='server') 

files_fetched = get(experiments, ['files_fetched'])

In [ ]:
df_mc_proxy = df_rx_proxy.filter(like='multicast')
df_mc_server = df_tx_server.filter(like='multicast')

df_mc = df_rx_proxy.filter(like='multicast').copy()
df_mc = df_mc.merge(df_mc_server, how='outer', left_index=True, right_index=True)
df_mc_reference = df_rx_proxy_reference.filter(like='multicast').copy()
df_mc_reference = df_mc_reference.merge(df_tx_server_reference.filter(like='multicast'), how='outer', left_index=True, right_index=True)
df_mc_no_fec = df_rx_proxy_no_fec.filter(like='multicast').copy()
df_mc_no_fec = df_mc_no_fec.merge(df_tx_server_no_fec.filter(like='multicast'), how='outer', left_index=True, right_index=True)
df_mc_fec = df_rx_proxy_fec.filter(like='multicast').copy()
df_mc_fec = df_mc_fec.merge(df_tx_server_fec.filter(like='multicast'), how='outer', left_index=True, right_index=True)

df_uc_proxy = df_rx_proxy.filter(like='unicast')
df_uc_server = df_tx_server.filter(like='unicast')

df_uc = df_uc_proxy.copy()
df_uc = df_uc.merge(df_uc_server, how='outer', left_index=True, right_index=True)
df_uc_reference = df_rx_proxy_reference.filter(like='unicast').copy()
df_uc_reference = df_uc_reference.merge(df_tx_server_reference.filter(like='unicast'), how='outer', left_index=True, right_index=True)
df_uc_no_fec = df_rx_proxy_no_fec.filter(like='unicast').copy()
df_uc_no_fec = df_uc_no_fec.merge(df_tx_server_no_fec.filter(like='unicast'), how='outer', left_index=True, right_index=True)
df_uc_fec = df_rx_proxy_fec.filter(like='unicast').copy()
df_uc_fec = df_uc_fec.merge(df_tx_server_fec.filter(like='unicast'), how='outer', left_index=True, right_index=True)

In [ ]:
tools.plot_data(df_mc_server, title='Multicast Mb', figsize=(30,7), ylabel='Mb')
tools.plot_data(df_uc_server, title='Unicast Mb', figsize=(30,7), ylabel='Mb')


In [ ]:
def take_window(df, df_total):
    temp = df_total.mask(df_total < 0)
    earliest_row = temp.idxmin().min()
    last_row = temp.idxmax().min()
    return df[earliest_row:last_row]

df2 = take_window(df_tx_server.filter(like='multicast'), df_tx_server.filter(like='multicast'))
df3 = take_window(df_rx_proxy.filter(like='multicast'), df_rx_proxy.filter(like='multicast'))

# Create a new series to store the results
series3 = pd.Series()

# Find the last experiment number
max_num = pd.Series(df3.index).str.extract("\['(\d+)',").astype(int).max()
# Loop through the odd numbers from 1 to 11
for i in range(1, max_num.values[0], 2):
    # Get the corresponding even number
    j = i + 1
    # Get the experiment names from the series
    exp1 = f"'{i}', 'proxy_interface_multicast']"
    exp2 = f"'{j}', 'proxy_interface_multicast']"
    # Calculate the value using the formula
    value = df3[exp2] / df2[exp1] - 1
    # Add the value to the new series with the experiment name
    series3[exp1] = f"{df2[exp1]:.3f} Mbps"
    series3[exp2] = f"{df3[exp2]:.3f} Mbps"
    series3[f"Exp {j} / Exp {i}"] = f"{value * 100:.2f} %"


series3

In [ ]:

tools.plot_data(df_rx_proxy.filter(like='multicast') / 1000000, title='Multicast Mb', figsize=(30,7), ylabel='Mb')
tools.plot_data(df_tx_server.filter(like='multicast') / 1000000, title='Multicast Mb', figsize=(30,7), ylabel='Mb')